# Neural Network Force-Free magnetic field extrapolation - NF2
<img src="https://github.com/RobertJaro/NF2/blob/main/images/logo.jpg?raw=true" width="150" height="150">

This notebook provides basic NF2 extrapolations of SHARP regions (SDO/HMI). Fill the form below to select your observation and start the extrapolation.

Run all sections below to start your non-linear force-free extrapolation. Extrapolations take about 6 hours to complete in Google Colab. All results are logged to Weights And Biases (https://wandb.ai). If you want to continue an interrupted run, fill the WandB section.

GitHub Page: https://github.com/RobertJaro/NF2


#Install NF2

In [ ]:
!pip install git+https://github.com/RobertJaro/NF2@v0.3.0

from nf2.data.download import download_SHARP_series
from nf2.extrapolate import run
# util functions
from datetime import datetime

# Data Download

Downloading data requires an active registration at JSOC. http://jsoc.stanford.edu/ajax/register_email.html

In [ ]:
#@title Download Credentials
jsoc_email = 'robert.jarolim@uni-graz.at' #@param {type:"string"}

In [ ]:
#@title Date of observation
year = 2017 #@param {type:"integer"}
month = 9 #@param {type:"integer"}
day = 6 #@param {type:"integer"}
hour = 8 #@param {type:"integer"}
minute = 36 #@param {type:"number"}

date = datetime(year, month, day, hour, minute)

In [ ]:
#@title Active Region
#@markdown specify either the NOAA or SHARP number of the active region
noaa_num = 12673 #@param {type:"number"}
sharp_num = None #@param {type:"number"}

download_dir = 'AR_12673' #@param {type:"string"}


In [ ]:
download_result = download_SHARP_series(download_dir=download_dir,
                                        email=jsoc_email, t_start=date,
                                        noaa_num=noaa_num, sharp_num=sharp_num)
download_result

# Extrapolation

In [ ]:
#@title Storage path
base_path = 'extrapolation_AR12673' #@param {type:"string"}

In [ ]:
#@title Data configuration
#@markdown select directory for saving extrapolation results
base_path = 'extrapolation_AR12673' #@param {type:"string"}

#@markdown insert downloaded SHARP files
br_path = '/content/AR_12673/hmi.sharp_cea_720s.7115.20170906_083600_TAI.Br.fits' #@param {type:"string"}
bt_path = '/content/AR_12673/hmi.sharp_cea_720s.7115.20170906_083600_TAI.Bt.fits' #@param {type:"string"}
bp_path = '/content/AR_12673/hmi.sharp_cea_720s.7115.20170906_083600_TAI.Bp.fits' #@param {type:"string"}

#@markdown insert downloaded SHARP error files (optional)
use_error_files = True #@param {type:"boolean"}
br_error_path = '/content/AR_12673/hmi.sharp_cea_720s.7115.20170906_083600_TAI.Br_err.fits' #@param {type:"string"}
bt_error_path = '/content/AR_12673/hmi.sharp_cea_720s.7115.20170906_083600_TAI.Bt_err.fits' #@param {type:"string"}
bp_error_path = '/content/AR_12673/hmi.sharp_cea_720s.7115.20170906_083600_TAI.Bp_err.fits' #@param {type:"string"}

data_config = {
  'type': 'fits',
  'slices': [{
      'fits_path' : {
        'Br': br_path,
        'Bt': bt_path,
        'Bp': bp_path
      }
  }],
  'iterations': 10000
}

if use_error_files:
  data_config['slices'][0]['error_path'] = {'Br_err': br_error_path,
                                         'Bt_err': bt_error_path,
                                         'Bp_err': bp_error_path}

In [ ]:
#@title Logging Settings
entity = "" #@param {type:"string"}
project = "nf2" #@param {type:"string"}
#@markdown Enter your credentials if you want to continue an interrupted run. This will restore the latest model and continue the training.
continue_training = False #@param {type:"boolean"}
id = "" #@param {type:"string"}

entity = None if entity == "" else entity
project = None if project == "" else project
logging_config = {'log_model': 'all', 'entity': entity, 'project': project}

if continue_training:
  logging_config['id'] = id

In [ ]:
#@title Advanced model settings
use_custom_settings = False #@param {type:"boolean"}
#@markdown Set `vector_potential` if you want to run a divergence-free extrapolation
vector_potential = False #@param {type:"boolean"}
#@markdown Change `dim` to set the available free parameters. Lower values reduce the runtime but may decrease the quality of the extrapolation. Use 256 for the standard setting and 64 for quick tests.
dim = 256 #@param {type:"number"}

if use_custom_settings:
  model_config = {'type': 'vector_potential' if vector_potential else 'b',
                  'dim': dim}
else:
  model_config = None

In [ ]:
#@title Advanced training settings
use_custom_settings = False #@param {type:"boolean"}
#@markdown Change the weighting of the force-free condition
lambda_ff = 0.1 #@param {type:"number"}
#@markdown Change the weighting of the divergence-free condition
lambda_div = 0.1 #@param {type:"number"}
#@markdown Change the weighting of the boundary condition
lambda_boundary = 1.0 #@param {type:"number"}



if use_custom_settings:
  boundary_loss = {'type': 'boundary', 'name': 'boundary', 'lambda': lambda_boundary,
                 'ds_id': ['boundary_01', 'potential']}
  force_free_loss = {'type': 'force_free', 'lambda': lambda_ff}
  divergence_free_loss = {'type': 'divergence', 'lambda': lambda_div}

  training_config = {'loss_config': [boundary_loss, force_free_loss, divergence_free_loss]}
else:
  training_config = {}

## Start Extrapolation
For the first run you will be redirected to the WandB login. Login with your existing user or register a new user to log your results online.

In [ ]:
run(base_path, data=data_config,
    logging=logging_config,
    training=training_config,
    model=model_config)